In [1]:
import cv2 as cv
import numpy as np

In [3]:
video_path = r"videos/bugs.mp4"
cap = cv.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()

    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    cv.imshow('frame',frame)
    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...


In [4]:
video_path = r"videos/bugs.mp4"
cap = cv.VideoCapture(video_path)
ret, first_frame = cap.read()
# Converts frame to grayscale because we only need the luminance channel for detecting edges - less computationally expensive
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)

# Create a mask image for drawing purposes
mask = np.zeros_like(first_frame)
# Sets image saturation to maximum
mask[..., 1] = 255

while(cap.isOpened()):
  ret, frame = cap.read()
  if not ret:
    print('No frames grabbed!')
    break

  cv.imshow('original frame',frame)
  gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
  # Calculates dense optical flow by Farneback method
  flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
  # Computes the magnitude and angle of the 2D vectors
  magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
  # Sets image hue according to the optical flow direction
  mask[..., 0] = angle * 180 / np.pi / 2
  # Sets image value according to the optical flow magnitude (normalized)
  mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
  # Converts HSV to RGB (BGR) color representation
  rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
  # Opens a new window and displays the output frame
  cv.imshow('dense optical flow', rgb)
  # Updates previous frame
  prev_gray = gray

  k = cv.waitKey(30) & 0xff
  if k == 27:
    break
  elif k == ord('s'):
    cv.imwrite('opticalfb.png', frame)
    cv.imwrite('opticalrgb.png', rgb)


# Frees up resources and closes all windows
cap.release()
cv.destroyAllWindows()

No frames grabbed!
